In [1]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yaml

In [ ]:
url = 'http://liquipedia.net/dota2/Players_(all)'
html = requests.get(url)
html.encoding = 'utf-8'
soup = BeautifulSoup(html.text, "lxml")
df = pd.DataFrame(columns = ['cog',
                             'player',
                             'name', 
                             'dob', 
                             'role', 
                             'team', 
                             'join',
                             'joinelo32',
                             'joinelo64',
                             'joinglicko',
                             'joinglickomu',
                             'joinglickophi',
                             'joinglickosigma',
                             'joinglicko2',
                             'joinglicko2mu',
                             'joinglicko2phi',
                             'joinglicko2sigma',                             
                             'leave',
                             'leaveelo32', 
                             'leaveelo64', 
                             'leaveglicko',
                             'leaveglickomu',
                             'leaveglickophi',
                             'leaveglickosigma',
                             'leaveglicko2',
                             'leaveglicko2mu',
                             'leaveglicko2phi',
                             'leaveglicko2sigma',                            
                            ])
tables = soup.find_all('tr')

In [ ]:
%%time
for table in tables[:]:
    try:
        string = re.findall(r'<a href="(.+)" title="(.+?)"', str(table))
        name = re.search(r'\w+\s\w+', str(table.find_all('td')[2].contents))
        if name is not None:
            name = name[0]
        else:
            name = 'N/A'
        url2 = 'http://wiki.teamliquid.net' + string[1][0]
        html2 = requests.get(url2)
        html2.encoding = 'utf-8'
        soup2 = BeautifulSoup(html2.text, "lxml")
        bd = soup2.find('span', class_='bday')   
        if bd is not None:
            bday = "".join([str(x) for x in bd.contents])
        else:
            continue
        hists = soup2.find_all('div', style="font-size:100%; line-height:-10px;")
        for hist in hists:
            date = hist.find('i').contents[0]
            join = date[:10]
            leave = date[13:]
            team = hist.find('a').contents[0]            
            join2 = join[-2:] + join[4:8] + join[:4]
            leave2 = leave[-2:] + leave[4:8] + leave[:4]
            urldat = 'http://www.datdota.com/api/ratings?date=' + join2
            htmldat = requests.get(urldat)
            htmldat.encoding = 'utf-8'
            elo = BeautifulSoup(htmldat.text, "lxml")
            elodict = yaml.load(elo.text)
            joinelolist = elodict['data']   
            #rating of leavedate from datdota and turn into list of dict
            if len(leave2) != 0:
                urldat = 'http://www.datdota.com/api/ratings?date=' + leave2
                htmldat = requests.get(urldat)
                htmldat.encoding = 'utf-8'
                elo = BeautifulSoup(htmldat.text, "lxml")
                elodict = yaml.load(elo.text)
                leaveelolist = elodict['data']
            else:
                leaveelolist = []

            teamnamelower = re.sub(r'\W+', '', team).lower()
            #init elo32,64
            joinelo32 = 'N/A'
            joinelo64 = 'N/A'
            joinglicko = 'N/A'
            joinglickomu = 'N/A'
            joinglickophi = 'N/A'
            joinglickosigma = 'N/A'
            joinglicko2 = 'N/A'
            joinglicko2mu = 'N/A'
            joinglicko2phi = 'N/A'
            joinglicko2sigma = 'N/A'
            leaveelo32 = 'N/A'
            leaveelo64 = 'N/A'
            leaveglicko = 'N/A'
            leaveglickomu = 'N/A'
            leaveglickophi = 'N/A'
            leaveglickosigma = 'N/A'
            leaveglicko2 = 'N/A'
            leaveglicko2mu = 'N/A'
            leaveglicko2phi = 'N/A'
            leaveglicko2sigma = 'N/A'
            #for each dict in list of dict of joinelo compare teamname to teamname from liquipedia
            for joinelo in joinelolist:
                joinelolower = re.sub(r'\W+', '', joinelo['teamName'].lower())
                if (((joinelolower in teamnamelower)
                or (teamnamelower in joinelolower))
                and (abs(len(teamnamelower) - len(joinelolower)) < 5)):
                    joinelo32 = joinelo['elo32']['current']
                    joinelo64 = joinelo['elo64']['current']
                    joinglicko = joinelo['glicko']['rating']
                    joinglickomu = joinelo['glicko']['mu']
                    joinglickophi = joinelo['glicko']['phi']
                    joinglickosigma = joinelo['glicko']['sigma']
                    joinglicko2 = joinelo['glicko2']['rating']
                    joinglicko2mu = joinelo['glicko2']['mu']
                    joinglicko2phi = joinelo['glicko2']['phi']
                    joinglicko2sigma = joinelo['glicko2']['sigma']

            for leaveelo in leaveelolist:
                leaveelolower = re.sub(r'\W+', '', leaveelo['teamName'].lower())
                if (((leaveelolower in teamnamelower)
                or (teamnamelower in leaveelolower))
                and (abs(len(teamnamelower) - len(leaveelolower)) < 5)):
                    leaveelo32 = leaveelo['elo32']['current']
                    leaveelo64 = leaveelo['elo64']['current']
                    leaveglicko = leaveelo['glicko']['rating']
                    leaveglickomu = leaveelo['glicko']['mu']
                    leaveglickophi = leaveelo['glicko']['phi']
                    leaveglickosigma = leaveelo['glicko']['sigma']
                    leaveglicko2 = leaveelo['glicko2']['rating']        
                    leaveglicko2mu = leaveelo['glicko2']['mu']        
                    leaveglicko2phi = leaveelo['glicko2']['phi']        
                    leaveglicko2sigma = leaveelo['glicko2']['sigma']        
                    
                    
            rls = soup2.find(text='Role(s):').findNext().find_all('a')
            roles = []
            for rl in rls:
                if rl is not None:
                    roles.append(rl.contents[0])
                else:
                    roles.append('N/A')
            for r in roles[::-1]:
                if r in ['Carry', 'Solo Middle', 'Offlaner']:
                    role = 1
                elif r in ['Support']:
                    role = 2
                elif not ((role == 1) or (role == 2)):
                    role = 3
                
            df = df.append({'cog': string[0][1], 
                            'player': string[1][1], 
                            'name': name,
                            'dob': bday, 
                            'role': role,
                            'team': team, 
                            'join': join,
                            'joinelo32': joinelo32, 
                            'joinelo64': joinelo64,
                            'joinglicko': joinglicko,
                            'joinglickomu': joinglickomu,
                            'joinglickophi': joinglickophi,
                            'joinglickosigma': joinglickosigma,
                            'joinglicko2': joinglicko,
                            'joinglicko2mu': joinglickomu,
                            'joinglicko2phi': joinglickophi,
                            'joinglicko2sigma': joinglickosigma,             
                            'leave': leave,
                            'leaveelo32': leaveelo32, 
                            'leaveelo64': leaveelo64,
                            'leaveglicko': leaveglicko,
                            'leaveglickomu': leaveglickomu,
                            'leaveglickophi': leaveglickophi,
                            'leaveglickosigma': leaveglickosigma,
                            'leaveglicko2': leaveglicko2,
                            'leaveglicko2mu': leaveglickomu,
                            'leaveglicko2phi': leaveglickophi,
                            'leaveglicko2sigma': leaveglickosigma,}, ignore_index=True)

    except IndexError:
        continue
        
    except AttributeError:
        continue